# Identify Label Errors with Label Inspector from AWS Marketplace 


Cleanlab's [Label Inspector](https://aws.amazon.com/marketplace/pp/prodview-rlbhc2lxttdio) automatically detects label errors in your classification dataset. You just need a (tabular or text) dataset containing class labels and feature values for each datapoint, and this solution will flag examples that are likely mislabeled.

This sample notebook demonstrates how to use the Label Inspector via Amazon SageMaker.

## Pre-requisites
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. Some hands-on experience using [Amazon SageMaker](https://aws.amazon.com/sagemaker/).
1. To use this algorithm successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [Label Inspector](https://aws.amazon.com/marketplace/pp/prodview-rlbhc2lxttdio). 

View our handy [AWS Marketplace Guide](../GUIDE.md) if you get stuck anywhere, especially with providing credentials/ARNs or other setup steps.

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to Label Inspector

To subscribe to the Label Inspector offering:
1. Open the AWS Marketplace listing page for [Label Inspector](https://aws.amazon.com/marketplace/pp/prodview-rlbhc2lxttdio).
1. On the listing,  click on **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you agree with the EULA and pricing terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn**. This is the algorithm ARN that you need to specify to use this algorithm. Copy the ARN corresponding to your region and specify it in the following cell.

> **Note**: This is a reference notebook and it cannot run unless you enter the algorithm ARN in the call below.

In [1]:
algo_arn = "<Specify the ARN for Label Inspector obtained from AWS Marketplace>"

## 2. Set up environment

In [2]:
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
import os
import pandas as pd
import tarfile

The two code cells below might have a different setup if you are running this sample notebook locally, please check out the [guide to run sample notebooks locally](../GUIDE.md/#run-sample-notebooks-locally) for more information.

In [3]:
try:
    session = boto3.Session()
    sagemaker_session = sage.Session(session)

except ValueError:
    # AWS access key id and secret access key only needs to be specified if running notebook locally 
    # (and AWS credentials were not previously setup)
    aws_access_key_id = "<Specify your AWS Access ID>"
    aws_secret_access_key = "<Specify your AWS Secret Access Key>"
    region = "us-east-1"  # replace with other region if you want, ensure that it matches the region in the ARN
    session = boto3.Session(aws_access_key_id, aws_secret_access_key, region_name=region)
    sagemaker_session = sage.Session(session)

In [4]:
# local variable only needs to be specified if running notebook locally rather than in Sagemaker
local_variable_for_sm_role = "arn:aws:iam::XXXXXX:role/service-role/SageMaker-XXXXX"  

try:
    role = get_execution_role()
except ValueError:
    role = local_variable_for_sm_role

In [5]:
# define S3 locations for saving data, replace if you would like to store your data in alternative locations
bucket = sagemaker_session.default_bucket()  # bucket where data will be stored
base_folder_name = "label-inspector"  # folder inside your bucket where data will be stored

instance_type = "ml.m5.xlarge"  # replace with "ml.m5.24xlarge" if more memory is required (big dataset) and to get results much faster (and potentially more accurate results)

## 3. Prepare dataset and Upload to Amazon S3 (skip if data is already in S3)

### View Sample Dataset 

Here is an example dataset that you can run Label Inspector on. Label inspector will take approximately 5 minutes to train a ML model and identify label errors on this example dataset.

If using your own data, please ensure that the first line of your CSV file is a header with the column names for your data. The first column of your data must be the containing the class labels (remaining columns will be treated as predictive features). Also make sure that the labels are categorical strings (e.g. not continuous numbers, but discrete integers are okay), as only multi-class (and binary) classification datasets are supported. 

In [6]:
dataset = pd.read_csv("data/input/dataset.csv")
dataset.head(5)

,letter_grade,exam_1,exam_2,exam_3,notes
0,C,53,77,93,NaN
1,B,81,64,80,great participation +10
2,B,74,88,97,NaN
3,C,61,94,78,NaN
4,C,48,90,91,NaN


### Upload datasets to Amazon S3

In [7]:
training_dataset = "data/input/dataset.csv"  # replace filepath here if using your own dataset

In [8]:
# upload data to S3
input_folder = "{}/{}".format(base_folder_name, "input")
training_data_location = sagemaker_session.upload_data(training_dataset, bucket=bucket, key_prefix=input_folder)

In [ ]:
# you can find your data here after uploading
training_data_location

## 4. Train a ML model to analyze the labels in our dataset

After ensuring that our data is an accessible Amazon S3 bucket (only read permissions are required for the algorithm to access the input data bucket), we are ready to train a machine learning model. This model can be automatically trained on diverse types of tabular/text data via state-of-the-art AutoML, and is used to identify which labels are likely incorrect. 

In the code cell below we specify the S3 location of our data. If you have followed the [Upload datasets to Amazon S3](#Upload-datasets-to-Amazon-S3) section of this tutorial, the S3 location should already be specified. If you are using your own dataset, make sure to specify the location of your data below.

In [10]:
training_data = training_data_location  # replace with the S3 URI of your data if using your own dataset

In [11]:
# formatting the output folder location based on variables specified above
# this is boilerplate code and does not need to be edited
output_location = "s3://{}/{}/{}".format(bucket, base_folder_name, "output")  

Next, we specify the hyperparameters for our algorithm. Currently, our algorithm only supports one hyperparameter `runtime` with two options:

- `fast` will have a shorter execution time, but may not produce the best quality results (maximum execution time: 2 hours, will take much less time for most datasets)
- `high_accuracy` will be slower, but produces high quality results (maximum execution time: 13 hours, will take much less time for most datasets)

In either case, you can get results faster by specifying a more powerful `instance_type` (and the results may be more accurate). When estimating costs, keep in mind that a more powerful instance can run the job faster (and has more memory available which may be required for larger datasets). ML training scales proportionally to the size of your dataset and may take some time, so just keep this job running and check back later to see if it has completed.

In [12]:
hyperparameters = {"runtime": "high_accuracy"}  # change to "fast" to get quicker results (will be less accurate)

Then, we create an estimator object for running a training job and train our model. For information on creating an `Estimator` object, check out the [documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html).

In [13]:
estimator = sage.algorithm.AlgorithmEstimator(
    algorithm_arn=algo_arn,
    base_job_name="label-inspector",
    role=role,
    instance_count=1,
    instance_type=instance_type,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters,
)

In [14]:
# run the training job
estimator.fit({"training": training_data})

INFO:sagemaker:Creating training-job with name: label-inspector-2023-03-15-22-43-22-258


2023-03-15 22:43:22 Starting - Starting the training job...
2023-03-15 22:43:37 Starting - Preparing the instances for training...
2023-03-15 22:44:18 Downloading - Downloading input data...
2023-03-15 22:44:39 Training - Downloading the training image.........
2023-03-15 22:46:19 Training - Training image download completed. Training in progress...Analyzing your data ... the estimated maximum runtime (upper bound) is: 13 hours

2023-03-15 22:47:50 Uploading - Uploading generated training modelLabel errors found.

2023-03-15 22:48:01 Completed - Training job completed
Training seconds: 222
Billable seconds: 222


In [ ]:
# your output will be available on following path
output_file_location = os.path.join(output_location, estimator._current_job_name, "output")
output_file_location

## 5. Fetch results of the label analysis

After the training job completes, we can get the results output by this analysis from the S3 bucket specified above. The main output of this solution is a CSV file with information about each label in your dataset. To learn more about what each column of the output file contains, check out our documentation [here](README.md#output). The examples flagged as likely mislabeled with the lowest label quality scores are the ones whose labels you should review closely.

In [16]:
s3 = session.client('s3')

# downloading the file from S3
with open('data/output/output.tar.gz', 'wb') as f:
    s3.download_fileobj(bucket, f"{base_folder_name}/output/{estimator._current_job_name}/output/output.tar.gz", f)

In [17]:
with tarfile.open('data/output/output.tar.gz') as file:
    file.extractall('data/output/')

In [18]:
cleanset = pd.read_csv("data/output/cleanset.csv")
cleanset

,is_label_issue,label_score,given_label,predicted_label
0,False,0.672778,C,C
1,False,0.691504,B,B
2,False,0.729446,B,B
3,False,0.662407,C,C
4,False,0.458495,C,C
...,...,...,...,...
936,False,0.377611,F,F
937,False,0.417643,F,F
938,False,0.733176,F,F
939,False,0.755515,B,B


Each row in the returned cleanset corresponds to a row in your original dataset. You can concatenate these two files into one to better review the identified label errors. We show an example of how to view the top errors in our sample dataset below:

In [19]:
original_dataset = pd.read_csv("data/input/dataset.csv")  # replace filepath here if using your own dataset
merged_cleanset = pd.concat([original_dataset, cleanset], axis=1)

Then, we can view the top label errors that was identified by filtering for the examples where `is_label_issue` is `True`, and sorting them by their `label_score` (lower score indicates more errorneous examples).

In [20]:
merged_cleanset[merged_cleanset["is_label_issue"] == True].sort_values("label_score").head()

,letter_grade,exam_1,exam_2,exam_3,notes,is_label_issue,label_score,given_label,predicted_label
318,F,98,51,74,NaN,True,0.012267,F,C
705,A,97,0,90,"cheated on exam, gets 0pts",True,0.014680,A,D
689,B,77,51,70,NaN,True,0.015814,B,D
597,F,81,100,74,NaN,True,0.018192,F,B
619,F,89,77,68,NaN,True,0.022600,F,C


## Other Tips

To get results faster (but potentially less accurate estimates): Specify a more powerful instance type and set `hyperparameters = {"runtime": "fast"}`. You can also try subsampling your dataset to a smaller one for a quick trial run (although be aware the ML training -- and hence accuracy of estimated label issues -- will become worse when the dataset is small). 

To get more accurate results: Specify a more powerful instance type. If you have text fields in your dataset (i.e. strings that are not discrete categories), use an instance that has a GPU (like a p-instance or g-instance) so large language models can be fine-tuned on your data. Also set `hyperparameters = {"runtime": "high_accuracy"}`.

To ask questions or report problems, please email: support@cleanlab.ai

To run a more in-depth analysis of your data and labels, try [Cleanlab Studio](https://cleanlab.ai/studio/) for free (it supports image data as well).

## Note about Real-time and Batch Transform

Currently, real-time and batch inference has no effect for the Label Inspector, do not try using it. All results are returned at the end of the training job. 

We have a beta-version of the Label Inspector that supports real-time and batch inference, either to get predictions from a robust classifier model (trained with label issues fixed) or to evaluate the label quality of future test datapoints on the fly. To request access, email: support@cleanlab.ai


## Clean Up Resources (Optional)

The code cell below contains the code that can be used to automatically delete the files created in S3 in this sample notebook. Be wary that this command will delete all the files stored in the `base_folder_name` folder specified above. Proceed with caution if you have previously stored other files in that folder.

Alternatively, you can navigate to the [S3 Management Console](https://s3.console.aws.amazon.com/) and manually delete these data files yourself.

In [21]:
# resource = session.resource('s3')
# resource_bucket = resource.Bucket(bucket)

# before executing the code below, ensure there is nothing else important in this folder
# resource_bucket.objects.filter(Prefix=f"{base_folder_name}/").delete()  